load all the pickle and model files 

In [5]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [3]:
model=load_model('model.h5')
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,947 (11.52 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [4]:
with open('labelencoder.pkl','rb') as file:
    labelencoder=pickle.load(file)

with open('onehotencoder.pkl','rb') as file:
    onehotencoder=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

labelencoder, onehotencoder, scaler

(LabelEncoder(), OneHotEncoder(), StandardScaler())

In [24]:
# example input data
input_data={
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000,
}

as we know from user input we first need to convert the gender and geography to numerical data and then all the input data we need to standardized

In [25]:
df=pd.DataFrame([input_data])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [26]:
df['Gender']=labelencoder.transform(df['Gender'])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [27]:
geo_df=onehotencoder.transform([df['Geography']]).toarray()
geo_df=pd.DataFrame(geo_df,columns=onehotencoder.get_feature_names_out(['Geography']))
geo_df

c:\Users\srish\anaconda3\envs\langchain\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [28]:
df=pd.concat([df.drop('Geography',axis=1),geo_df],axis=1)
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [31]:
scaled_df=scaler.transform(df)
scaled_df

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [32]:
prediction=model.predict(scaled_df)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step


array([[0.01365606]], dtype=float32)

In [33]:
prediction_probabilty=prediction[0][0]
prediction_probabilty

0.013656064